# 第3章：P2SH 脚本工程

> **参考**: `book/translations/zh-Hans/Chapter 3.md`  
> **代码示例**: `code/chapter03/`  
> **最后更新**: 2025-12-06

---

Bitcoin 的第一个真正的可编程性出现在支付到脚本哈希（Pay-to-Script-Hash，P2SH）中，复杂的花费条件优雅地隐藏在简单的 20 字节哈希后面。本章连接基础 P2PKH 交易和 Taproot 复杂脚本树之间的桥梁，展示 Bitcoin 的脚本系统如何支持企业资金管理和时间锁定继承规划等实际应用。


## 3.1 P2SH 架构：哈希背后的脚本

P2SH 使任何脚本都可以用紧凑的 20 字节哈希表示，将脚本复杂性从 UTXO 集转移到花费时。

### 两阶段验证模型

P2SH 通过两个不同阶段运行：

**阶段 1：哈希验证**
```
OP_HASH160 <script_hash> OP_EQUAL
```

**阶段 2：脚本执行**
```
<revealed_script> → Execute as Bitcoin Script
```

### P2SH 地址生成过程

P2SH 遵循第一章中介绍的 Hash160 → Base58Check 模式，但哈希的是脚本而非公钥：

```
Script Serialization → hex_encoded_script
Hash160(script)     → 20_bytes_script_hash  
Version + Base58Check → 3...address (mainnet)
```

所有 P2SH 地址在主网上以"3"开头，在测试网上以"2"开头，立即将它们与 P2PKH 地址区分开来。

### ScriptSig 构建模式

P2SH 的解锁脚本（ScriptSig）遵循特定模式：

```
<script_data> <serialized_redeem_script>
```

其中 `<script_data>` 包含满足赎回脚本条件所需的值，`<serialized_redeem_script>` 是哈希与锁定脚本匹配的原始脚本。


## 3.2 多重签名资金库：2-of-3 企业安全

企业 Bitcoin 托管通常需要多方授权以防止单点故障。2-of-3 多重签名方案确保没有单个人可以单方面访问资金，同时保持操作灵活性。

### 业务场景：初创公司资金管理

考虑一个有三个关键利益相关者的区块链初创公司：
- **Alice**：具有运营权限的 CEO
- **Bob**：具有技术监督的 CTO  
- **Carol**：具有财务控制的 CFO

他们的资金政策要求任何资金流动都需要两个签名，既防止单人风险，又不需要一致共识。


In [1]:
# 示例 1: 创建多重签名 P2SH
# 参考: code/chapter03/01_create_multisig_p2sh.py

from bitcoinutils.setup import setup
from bitcoinutils.keys import PrivateKey
from bitcoinutils.script import Script
from bitcoinutils.keys import P2shAddress


def create_multisig_p2sh():
    """Create a 2-of-3 multi-signature P2SH address"""
    setup('testnet')
    
    # Stakeholder public keys
    alice_pk = '02898711e6bf63f5cbe1b38c05e89d6c391c59e9f8f695da44bf3d20ca674c8519'
    bob_pk = '0284b5951609b76619a1ce7f48977b4312ebe226987166ef044bfb374ceef63af5'
    carol_pk = '0317aa89b43f46a0c0cdbd9a302f2508337ba6a06d123854481b52de9c20996011'
    
    # 2-of-3 multisig redeem script
    redeem_script = Script([
        'OP_2',           # Require 2 signatures
        alice_pk,         # Alice's public key
        bob_pk,           # Bob's public key  
        carol_pk,         # Carol's public key
        'OP_3',           # Total of 3 keys
        'OP_CHECKMULTISIG' # Multisig verification
    ])
    
    # Generate P2SH address
    p2sh_addr = P2shAddress.from_script(redeem_script)
    
    print(f"Redeem Script: {redeem_script.to_hex()}")
    print(f"P2SH Address: {p2sh_addr.to_string()}")
    
    return p2sh_addr, redeem_script


if __name__ == "__main__":
    create_multisig_p2sh()


Redeem Script: 522102898711e6bf63f5cbe1b38c05e89d6c391c59e9f8f695da44bf3d20ca674c8519210284b5951609b76619a1ce7f48977b4312ebe226987166ef044bfb374ceef63af5210317aa89b43f46a0c0cdbd9a302f2508337ba6a06d123854481b52de9c2099601153ae
P2SH Address: 2NDSSi5n5knjFqNMQMxyCezn6i18UQEh8Nj


### bitcoinutils 函数分析

**`Script([...])` 构造函数**：从 opcode 和数据列表创建 Script 对象。库自动处理 opcode（如 `'OP_2'`）到字节表示（`0x52`）的编码。

**`P2shAddress.from_script(script)`**：通过以下方式生成 P2SH 地址：
1. 将脚本序列化为字节
2. 计算 Hash160(script) 
3. 添加版本字节（主网为 0x05，测试网为 0xc4）
4. 应用 Base58Check 编码

**脚本序列化**：赎回脚本序列化为：
`522102898711e6bf63f5cbe1b38c05e89d6c391c59e9f8f695da44bf3d20ca674c8519210284b5951609b76619a1ce7f48977b4312ebe226987166ef044bfb374ceef63af5210317aa89b43f46a0c0cdbd9a302f2508337ba6a06d123854481b52de9c2099601153ae`

分解如下：
- `52`：OP_2
- `21`：推送 33 字节（压缩公钥）
- `02898711...`：Alice 的公钥
- `21`：推送 33 字节  
- `0284b595...`：Bob 的公钥
- `21`：推送 33 字节
- `0317aa89...`：Carol 的公钥
- `53`：OP_3
- `ae`：OP_CHECKMULTISIG


In [2]:
# 示例 2: 花费多重签名 P2SH
# 参考: code/chapter03/02_spend_multisig_p2sh.py

from bitcoinutils.setup import setup
from bitcoinutils.utils import to_satoshis
from bitcoinutils.transactions import Transaction, TxInput, TxOutput
from bitcoinutils.keys import PrivateKey, P2pkhAddress
from bitcoinutils.script import Script


def spend_multisig_p2sh():
    """Spend a 2-of-3 multi-signature P2SH UTXO"""
    setup('testnet')
    
    # Private keys for Alice and Bob (2-of-3)
    # These must match the keys used in create_multisig_p2sh.py
    alice_sk = PrivateKey('cPeon9fBsW2BxwJTALj3hGzh9vm8C52Uqsce7MzXGS1iFJkPF4AT')
    bob_sk = PrivateKey('cSNdLFDf3wjx1rswNL2jKykbVkC6o56o5nYZi4FUkWKjFn2Q5DSG')
    
    # Public keys (same as in create_multisig_p2sh)
    alice_pk = '02898711e6bf63f5cbe1b38c05e89d6c391c59e9f8f695da44bf3d20ca674c8519'
    bob_pk = '0284b5951609b76619a1ce7f48977b4312ebe226987166ef044bfb374ceef63af5'
    carol_pk = '0317aa89b43f46a0c0cdbd9a302f2508337ba6a06d123854481b52de9c20996011'
    
    # Recreate the redeem script (must match the one used to create the P2SH address)
    redeem_script = Script([
        'OP_2',
        alice_pk,
        bob_pk,
        carol_pk,
        'OP_3',
        'OP_CHECKMULTISIG'
    ])
    
    # Previous UTXO details
    utxo_txid = '4b869865bc4a156d7e0ba14590b5c8971e57b8198af64d88872558ca88a8ba5f'
    utxo_vout = 0
    utxo_amount = 0.00001600  # 1,600 satoshis
    
    # Recipient address (example)
    recipient_address = P2pkhAddress('myYHJtG3cyoRseuTwvViGHgP2efAvZkYa4')
    
    # Create transaction
    txin = TxInput(utxo_txid, utxo_vout)
    txout = TxOutput(to_satoshis(0.00000888), recipient_address.to_script_pub_key())
    tx = Transaction([txin], [txout])
    
    # Sign with Alice and Bob's keys
    alice_sig = alice_sk.sign_input(tx, 0, redeem_script)
    bob_sig = bob_sk.sign_input(tx, 0, redeem_script)
    
    # Construct ScriptSig: OP_0 <sig1> <sig2> <redeem_script>
    # OP_0 is required due to OP_CHECKMULTISIG bug
    txin.script_sig = Script([
        'OP_0',                    # OP_CHECKMULTISIG bug workaround
        alice_sig,                 # First signature
        bob_sig,                   # Second signature  
        redeem_script.to_hex()     # Reveal the redeem script
    ])
    
    # Get the signed transaction
    signed_tx = tx.serialize()
    
    print(f"Signed transaction: {signed_tx}")
    print(f"Transaction size: {tx.get_size()} bytes")
    
    return signed_tx


if __name__ == "__main__":
    spend_multisig_p2sh()


Signed transaction: 02000000015fbaa888ca582587884df68a19b8571e97c8b59045a10b7e6d154abc6598864b00000000fc004730440220694f08c86a34756928d4deb7c077a97ddec7cc5bb9a794c32d6bba98c12528c402205db0cf68b1584b02c313a3ed612f185ec559c35a875e47ba1e7f07aad79f7a6501473044022065f8c689be1ba7effab6026e50fb14bde8535d5c1e06a13f1789b7f187fbbbf902202dae9e9c4172545bcf0982c0a5f7dd41ff1bc70e586312002d0a16acca86fd9e014c69522102898711e6bf63f5cbe1b38c05e89d6c391c59e9f8f695da44bf3d20ca674c8519210284b5951609b76619a1ce7f48977b4312ebe226987166ef044bfb374ceef63af5210317aa89b43f46a0c0cdbd9a302f2508337ba6a06d123854481b52de9c2099601153aefdffffff0178030000000000001976a914c5b28d6bba91a2693a9b1876bcd3929323890fb288ac00000000
Transaction size: 337 bytes


### bitcoinutils 签名函数

**`private_key.sign_input(tx, input_index, script)`**：使用提供的脚本进行签名哈希计算，为特定交易输入创建 ECDSA 签名。对于 P2SH 输入，script 参数应该是赎回脚本。

**`script.to_hex()`**：将 Script 对象序列化为十六进制字节表示，在脚本执行期间作为数据推送到栈上。

### 多重签名栈执行分析

让我们使用真实交易数据追踪完整的脚本执行，理解 Bitcoin Core 的两阶段 P2SH 执行机制：

**交易 ID**：`e68bef534c7536300c3ae5ccd0f79e031cab29d262380a37269151e8ba0fd4e0`

## 阶段 1：ScriptSig + ScriptPubKey 执行

**初始状态：**
```
│ (empty)                                │
└────────────────────────────────────────┘
```

### 1. PUSH OP_0：多重签名 bug 修复
Bitcoin 的 OP_CHECKMULTISIG 有一个已知的 off-by-one bug，会从栈中多弹出一个项。推送一个 OP_0 来补偿。

```
│ 00 (op_zero)                           │
└────────────────────────────────────────┘
```

### 2. PUSH Alice 的签名：第一次授权
```
│ 30440220694f...7a6501 (alice_sig)      │
│ 00 (op_zero)                           │
└────────────────────────────────────────┘
```

### 3. PUSH Bob 的签名：第二次授权  
```
│ 3044022065f8...fd9e01 (bob_sig)        │
│ 30440220694f...7a6501 (alice_sig)      │
│ 00 (op_zero)                           │
└────────────────────────────────────────┘
```

### 4. PUSH 赎回脚本：揭示花费条件
```
│ 522102898711...601153ae (redeem_script)  │
│ 3044022065f8...fd9e01 (bob_sig)          │
│ 30440220694f...7a6501 (alice_sig)        │
│ 00 (op_zero)                             │
└──────────────────────────────────────────┘
```

### 5. OP_HASH160：验证脚本哈希匹配
执行 P2SH 锁定脚本 `OP_HASH160 <script_hash> OP_EQUAL`：

```
│ dd81b5beb3d8...5cb0ca (computed_hash)    │
│ 3044022065f8...fd9e01 (bob_sig)          │
│ 30440220694f...7a6501 (alice_sig)        │
│ 00 (op_zero)                             │
└──────────────────────────────────────────┘
```

### 6. PUSH 预期哈希：来自锁定脚本
```
│ dd81b5beb3d8...5cb0ca (expected_hash)    │
│ dd81b5beb3d8...5cb0ca (computed_hash)    │
│ 3044022065f8...fd9e01 (bob_sig)          │
│ 30440220694f...7a6501 (alice_sig)        │
│ 00 (op_zero)                             │
└──────────────────────────────────────────┘
```

### 7. OP_EQUAL：确认哈希匹配
```
│ 1 (true)                               │
│ 3044022065f8...fd9e01 (bob_sig)        │
│ 30440220694f...7a6501 (alice_sig)      │
│ 00 (op_zero)                           │
└────────────────────────────────────────┘
```

**（阶段 1 完成：哈希验证成功）**

## P2SH 转换：栈重置机制

**关键点**：Bitcoin Core 识别 P2SH 模式并通过以下方式转换到第二个验证阶段：

1. **检测 P2SH 模式**：`OP_HASH160 <hash> OP_EQUAL` 
2. **重置栈**：回到 post-scriptSig 状态（丢弃 TRUE）
3. **提取赎回脚本**：从原始 scriptSig 数据
4. **准备干净执行**：用于带签名数据的赎回脚本

**栈重置到 Post-ScriptSig 状态：**
```
│ 3044022065f8...fd9e01 (bob_sig)        │
│ 30440220694f...7a6501 (alice_sig)      │
│ 00 (op_zero)                           │
└────────────────────────────────────────┘
```

**（TRUE 被丢弃——赎回脚本从干净栈开始）**

## 阶段 2：赎回脚本执行

Bitcoin Core 现在执行赎回脚本：`OP_2 alice_pk bob_pk carol_pk OP_3 OP_CHECKMULTISIG`

### 8. OP_2：推送所需签名数量
```
│ 2 (required_sigs)                      │
│ 3044022065f8...fd9e01 (bob_sig)        │
│ 30440220694f...7a6501 (alice_sig)      │
│ 00 (op_zero)                           │
└────────────────────────────────────────┘
```

### 9-11. PUSH 公钥：加载验证密钥
```
│ 0317aa89b43f...996011 (carol_pk)       │
│ 0284b5951609...eef63af5 (bob_pk)       │
│ 02898711e6bf...674c8519 (alice_pk)     │
│ 2 (required_sigs)                      │
│ 3044022065f8...fd9e01 (bob_sig)        │
│ 30440220694f...7a6501 (alice_sig)      │
│ 00 (op_zero)                           │
└────────────────────────────────────────┘
```

### 12. OP_3：推送总密钥数量
```
│ 3 (total_keys)                         │
│ 0317aa89b43f...996011 (carol_pk)       │
│ 0284b5951609...eef63af5 (bob_pk)       │
│ 02898711e6bf...674c8519 (alice_pk)     │
│ 2 (required_sigs)                      │
│ 3044022065f8...fd9e01 (bob_sig)        │
│ 30440220694f...7a6501 (alice_sig)      │
│ 00 (op_zero)                           │
└────────────────────────────────────────┘
```

### 13. OP_CHECKMULTISIG：验证签名
opcode 消耗：
- 密钥数量（3）
- 公钥（Alice、Bob、Carol）  
- 签名数量（2）
- 签名（Alice 的、Bob 的）
- 额外项（OP_0，由于 bug）

验证过程：
1. Alice 的签名针对 Alice 的公钥验证 ✓
2. Bob 的签名针对 Bob 的公钥验证 ✓  
3. 满足所需阈值（2-of-3）✓

### 最终状态：多重签名验证完成
```
│ 1 (true)                               │
└────────────────────────────────────────┘
```

**（P2SH 执行成功：干净的两阶段验证）**


## 3.3 时间锁定继承：CSV 增强的 P2SH

检查序列验证（CheckSequenceVerify，CSV）支持相对时间锁，其中花费相对于 UTXO 创建时间延迟。让我们使用实际测试网数据检查真实实现。

### 真实实现：3 区块时间锁

**交易 ID**：`34f5bf0cf328d77059b5674e71442ded8cdcfc723d0136733e0dbf180861906f`

此交易演示了一个将 CSV 时间锁与 P2PKH 签名验证结合的 P2SH 脚本——继承和托管应用的常见模式。


In [4]:
# 示例 3: 创建 CSV 时间锁脚本
# 参考: code/chapter03/03_create_csv_script.py

from bitcoinutils.setup import setup
from bitcoinutils.keys import PrivateKey, P2pkhAddress
from bitcoinutils.transactions import Sequence
from bitcoinutils.constants import TYPE_RELATIVE_TIMELOCK
from bitcoinutils.script import Script
from bitcoinutils.keys import P2shAddress


def create_csv_script():
    """Create a CSV time-locked P2SH script (3 blocks delay)"""
    setup('testnet')
    
    private_key = PrivateKey('cRxebG1hY6vVgS9CSLNaEbEJaXkpZvc6nFeqqGT7v6gcW7MbzKNT')
    public_key = private_key.get_public_key()
    p2pkh_addr = public_key.get_address()
    
    relative_blocks = 3
    seq = Sequence(TYPE_RELATIVE_TIMELOCK, relative_blocks)
    
    redeem_script = Script([
        seq.for_script(),
        'OP_CHECKSEQUENCEVERIFY',
        'OP_DROP',
        'OP_DUP',
        'OP_HASH160',
        p2pkh_addr.to_hash160(),
        'OP_EQUALVERIFY', 
        'OP_CHECKSIG'
    ])
    
    p2sh_addr = P2shAddress.from_script(redeem_script)
    
    print(f"Public Key: {public_key.to_hex()}")
    print(f"P2PKH Address: {p2pkh_addr.to_string()}")
    print(f"Redeem Script: {redeem_script.to_hex()}")
    print(f"P2SH Address: {p2sh_addr.to_string()}")
    print(f"Time Lock: {relative_blocks} blocks")
    
    return p2sh_addr, redeem_script, private_key, public_key


if __name__ == "__main__":
    create_csv_script()


Public Key: 0250be5fc44ec580c387bf45df275aaa8b27e2d7716af31f10eeed357d126bb4d3
P2PKH Address: moyxCWpvEg9w5gGGV3XVEGwQXttDdtoEkc
Redeem Script: 53b27576a9145cdc28533660ee003aadd5bbd883196cab2ff90288ac
P2SH Address: 2N4Lnv72RRSJzz21PTnsBeECZHuMmVyS3iG
Time Lock: 3 blocks


### bitcoinutils CSV 函数

**`Sequence(TYPE_RELATIVE_TIMELOCK, blocks)`**：为基于区块的相对延迟创建序列对象。序列值编码将由 OP_CHECKSEQUENCEVERIFY 强制执行的时间约束。

**`seq.for_script()`**：返回格式化为脚本 opcode 使用的序列值（将延迟值推送到栈上）。

**`seq.for_input_sequence()`**：返回交易输入序列字段的序列值，CSV 将对此进行验证。


In [5]:
# 示例 4: 花费 CSV 时间锁脚本
# 参考: code/chapter03/04_spend_csv_script.py

from bitcoinutils.setup import setup
from bitcoinutils.utils import to_satoshis
from bitcoinutils.transactions import Transaction, TxInput, TxOutput, Sequence
from bitcoinutils.keys import PrivateKey, P2pkhAddress
from bitcoinutils.constants import TYPE_RELATIVE_TIMELOCK
from bitcoinutils.script import Script


def spend_csv_script():
    """Spend a CSV time-locked P2SH UTXO"""
    setup('testnet')
    
    private_key = PrivateKey('cRxebG1hY6vVgS9CSLNaEbEJaXkpZvc6nFeqqGT7v6gcW7MbzKNT')
    public_key = private_key.get_public_key()
    p2pkh_addr = public_key.get_address()
    
    relative_blocks = 3
    seq = Sequence(TYPE_RELATIVE_TIMELOCK, relative_blocks)
    
    redeem_script = Script([
        seq.for_script(),
        'OP_CHECKSEQUENCEVERIFY',
        'OP_DROP',
        'OP_DUP',
        'OP_HASH160',
        p2pkh_addr.to_hash160(),
        'OP_EQUALVERIFY',
        'OP_CHECKSIG'
    ])
    
    utxo_txid = '34f5bf0cf328d77059b5674e71442ded8cdcfc723d0136733e0dbf180861906f'
    utxo_vout = 0
    recipient_address = P2pkhAddress('myYHJtG3cyoRseuTwvViGHgP2efAvZkYa4')
    
    txin = TxInput(utxo_txid, utxo_vout, sequence=seq.for_input_sequence())
    txout = TxOutput(to_satoshis(0.00001), recipient_address.to_script_pub_key())
    tx = Transaction([txin], [txout])
    
    sig = private_key.sign_input(tx, 0, redeem_script)
    txin.script_sig = Script([
        sig,
        public_key.to_hex(),
        redeem_script.to_hex()
    ])
    
    signed_tx = tx.serialize()
    
    print(f"Sequence value: {seq.for_input_sequence()}")
    print(f"Signed transaction: {signed_tx}")
    print(f"Transaction size: {tx.get_size()} bytes")
    
    return signed_tx


if __name__ == "__main__":
    spend_csv_script()


Sequence value: b'\x03\x00\x00\x00'
Signed transaction: 02000000016f90610818bf0d3e7336013d72fcdc8ced2d44714e67b55970d728f30cbff53400000000874730440220745857374e7c6f798e50ccadc39cad0e4759d99e48f113e562e9b3310b9ffb8a02201d80be6b5869de3cabfd5ac40494d37f49629089aa20a694645d163dcb65b74b01210250be5fc44ec580c387bf45df275aaa8b27e2d7716af31f10eeed357d126bb4d31c53b27576a9145cdc28533660ee003aadd5bbd883196cab2ff90288ac0300000001e8030000000000001976a914c5b28d6bba91a2693a9b1876bcd3929323890fb288ac00000000
Transaction size: 220 bytes


### CSV 栈执行分析

让我们使用测试网示例的真实交易数据追踪执行：

**ScriptSig 数据**：
- 签名：`30440220...`（71 字节）
- 公钥：`0250be5fc44ec580c387bf45df275aaa8b27e2d7716af31f10eeed357d126bb4d3`（33 字节）  
- 赎回脚本：`53b27576a9145cdc...88ac`（28 字节）

## 阶段 1：P2SH 哈希验证

**（应用栈重置机制——详见多重签名部分）**

## 阶段 2：CSV + P2PKH 执行

**初始状态**（P2SH 重置后）：
```
│ 0250be5fc44ec...4d3 (pubkey)           │
│ 30440220a1b2...c3d401 (signature)      │
└────────────────────────────────────────┘
```

### 1. PUSH 3：时间延迟要求
```
│ 3 (delay_blocks)                       │
│ 0250be5fc44ec...4d3 (pubkey)           │
│ 30440220a1b2...c3d401 (signature)      │
└────────────────────────────────────────┘
```

### 2. OP_CHECKSEQUENCEVERIFY：验证时间锁
CSV 验证交易输入的序列号 ≥ 3：

```
│ 3 (delay_blocks)                       │
│ 0250be5fc44ec...4d3 (pubkey)           │
│ 30440220a1b2...c3d401 (signature)      │
└────────────────────────────────────────┘
```

**（验证：nSequence ≥ UTXO 创建后 3 个区块）**

### 3. OP_DROP：移除延迟值
```
│ 0250be5fc44ec...4d3 (pubkey)           │
│ 30440220a1b2...c3d401 (signature)      │
└────────────────────────────────────────┘
```

### 4. OP_DUP：开始 P2PKH 验证
```
│ 0250be5fc44ec...4d3 (pubkey)           │
│ 0250be5fc44ec...4d3 (pubkey)           │
│ 30440220a1b2...c3d401 (signature)      │
└────────────────────────────────────────┘
```

### 5. OP_HASH160：哈希公钥
```
│ 5cdc28d6b1876...cabaadcc (pubkey_hash) │
│ 0250be5fc44ec...4d3 (pubkey)           │
│ 30440220a1b2...c3d401 (signature)      │
└────────────────────────────────────────┘
```

### 6. PUSH 预期哈希：来自赎回脚本
```
│ 5cdc28d6b1876...cabaadcc (expected_hash) │
│ 5cdc28d6b1876...cabaadcc (computed_hash) │
│ 0250be5fc44ec...4d3 (pubkey)             │
│ 30440220a1b2...c3d401 (signature)        │
└──────────────────────────────────────────┘
```

### 7. OP_EQUALVERIFY：确认哈希匹配
```
│ 0250be5fc44ec...4d3 (pubkey)           │
│ 30440220a1b2...c3d401 (signature)      │
└────────────────────────────────────────┘
```

### 8. OP_CHECKSIG：最终签名验证
```
│ 1 (true)                               │
└────────────────────────────────────────┘
```

**（时间锁满足且签名验证——CSV 花费成功）**

### 时间锁错误处理

**常见错误：`non-BIP68-final`**

如果在时间锁过期前尝试花费：



交易被拒绝，因为 `nSequence < required_delay`，违反了 CSV 约束。

### CSV 应用

**数字继承**：资产在所有者不活跃指定时间后自动对继承人可访问。
**业务连续性**：企业资金可以包含用于运营紧急情况的自动释放机制。
**支付通道**：Lightning Network 使用 CSV 强制执行结算延迟，支持争议解决期。


## 3.4 P2SH vs P2PKH：脚本复杂性的演化

P2SH 将 Bitcoin Script 从简单的单签名授权扩展到复杂的多方和时间条件，同时保持相同的紧凑地址格式。

### P2SH 的固有局限性

然而，虽然 P2SH 通过用紧凑哈希替换显式脚本来提高效率，但它仍然在花费时暴露完整的赎回脚本。这意味着每个可能的条件——无论采用哪个分支——都必须在花费资金时被揭示。无法选择性地只揭示相关分支。
这种设计使 P2SH 在结构上本质上是线性和不透明的。与 Taproot 不同，它无法表达脚本树或使用 Merkle 分支隐藏未使用的逻辑。每个签名路径、时间锁条款或回退条件都完全在链上暴露。
此外，由于赎回脚本必须包含在 scriptSig 中，P2SH 交易在输入大小方面有显著开销。这导致更高的费用和更低的可扩展性，特别是对于多重签名或继承设置。
Taproot 通过允许复杂脚本在需要之前完全保持隐藏，并将它们嵌入到树结构中，只揭示执行的路径，直接解决了这些局限性。


## 章节总结

本章通过探索 P2SH 的两个基本模式——多重签名授权和时间锁定条件——连接了基础 P2PKH 交易和 Taproot 高级功能之间的桥梁。

**掌握的关键概念：**

**两阶段验证**：P2SH 的哈希然后执行模型为 Taproot 的承诺方案提供了概念基础，其中复杂脚本在花费之前保持私有。

**多方授权**：2-of-3 多重签名模式展示了 Bitcoin Script 如何处理条件逻辑和多个验证要求——理解 Taproot 脚本树执行所必需的技能。

**时间约束**：基于 CSV 的时间锁引入了相对时间概念，这些概念支撑了 Lightning Network 和其他建立在 Taproot 基础上的 Layer 2 协议。

**基于栈的编程**：多重签名和时间锁场景的详细栈执行追踪提供了调试和优化 Taproot 脚本路径所需的分析技能。

**bitcoinutils 熟练度**：Script 构建、P2shAddress 生成和签名创建的实践经验为开发者准备 Taproot 更复杂的原语。

**真实交易分析**：使用实际测试网交易和 mempool 数据构建生产 Taproot 开发所需的经验技能。

从 P2PKH 的简单签名验证到 P2SH 的复杂条件逻辑的进展，为 Taproot 的革命性方法奠定了基础：使复杂的智能合约与简单支付无法区分，同时提供前所未有的脚本灵活性和隐私性。

在下一章中，我们将研究 SegWit 的见证结构如何革命性地改变交易可延展性和费用计算——这些概念直接支持 Taproot 的效率改进，并构成理解 P2TR 基于见证的花费路径的基础。
